# Cover Type Prediction using ensembles

## Dataset Description
The dataset represents the data about trees which were planted in the US. The dataset consists of the information about 500000 trees. Your aim is to build  Random Forest Ensemble to predict the cover type of trees. In order to successfully complete this assignment you have to follow this algorithm:
* Load the training data
* Transform categorical features into vector representations
* Split dataset into the train and validation part
* Fit the Random Forest Ensemble into the training set
* Compare the accuracy of the fitted model with the Logistic Regression Model, which is about 0.67 for this set


If you have enough time, it will be very interesting to dig into further research through these steps:
* Determine which features are valuable for your model (calculate feature importance of your model).
* Try to reduce number of trees and see the results.
* Understand why the linear models have poor performance on this dataset.


## Loading data

Init pyspark session

In [1]:
from __future__ import division, print_function, unicode_literals # For the compatibility with Python 2

In [2]:
from pyspark.sql import SparkSession
spark_session = (
    SparkSession
    .builder
    .enableHiveSupport()
    .appName("spark sql")
    .master("local[4]")
    .getOrCreate()
)

Load train dataset located at /data/covertype2 with at least 60 partitions (use function repartition for this case). Use option `inferSchema` to save numerical features.

In [3]:
training_data = spark_session.read.load(
    "/data/covertype2/train.csv",
    format='com.databricks.spark.csv', 
    header=True, 
    inferSchema=True
).repartition(60).cache()

## Transforming data

As you can see, there are two categorical features in dataset: 'Soil_Type' and 'Wild_Type'. You have to transform them into the vector embeddings.

First of all, you have to use StringIndexer to transform feature types to indexes

In [4]:
from pyspark.ml.feature import StringIndexer

In [5]:
soil_indexer = StringIndexer(inputCol="Soil_Type", outputCol="soilIndexed")

In [6]:
wild_indexer = StringIndexer(inputCol="Wild_Type", outputCol="wildIndexed")

Apply OneHotEncoder technique to the dataset in order to get vectors for the Random Forest classification

In [7]:
from pyspark.ml.feature import OneHotEncoder

In [8]:
soil_encoder = OneHotEncoder(inputCol="soilIndexed", outputCol="SoilEncoder")

In [9]:
wild_encoder = OneHotEncoder(inputCol="wildIndexed", outputCol="WildEncoder")

Use the VectorAssembler technique to accumulate all features into one vector. Don't forget to use features that you have generated

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
vector_assembler = VectorAssembler(inputCols=[
    'SoilEncoder', # feature name of Soil type encoded
    'WildEncoder', # feature name of Wild type encoded
    'Elevation',
    'Aspect',
    'Slope',
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
], outputCol='features')

## Training

Fit the Random Forest model to the train dataset. Don't forget to split dataset into two parts to check your trained models. It is desirable to use about 100 trees with depth about 7 in order to avoid wasting too much time waiting while your model will be fit to the data. Try to adjust the options 'subsamplingRate' and 'featureSubsetStrategy' to get better results

<b> Extra task.</b> Use the Cross-Validation to check your model.

In [12]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

In [13]:
rf_classifer = RandomForestClassifier(
    labelCol="Target", featuresCol="features", numTrees=10, maxDepth=7,
    featureSubsetStrategy='all', subsamplingRate=0.1
)
# auto, all, sqrt, log2, onethird

In [14]:
param_grid = (
    ParamGridBuilder()
    .addGrid(rf_classifer.maxDepth, [1, 2, 6, 7, 8])
    # .addGrid(rf_classifer.featureSubsetStrategy, ['auto', 'onethird', 'sqrt'])
    .addGrid(rf_classifer.subsamplingRate, [0.1, 0.3, 0.5, 0.7, 0.9, 1.0])
    .build()
)

evaluator = MulticlassClassificationEvaluator(
    labelCol="Target", predictionCol="prediction", metricName="accuracy")

In [15]:
pipeline = Pipeline(stages=[
    soil_indexer, wild_indexer, soil_encoder, wild_encoder, vector_assembler, rf_classifer])

In [16]:
cross_val = CrossValidator(
    estimator=pipeline, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=15)

In [17]:
rf_model = cross_val.fit(training_data)

In [18]:
# rf_model.avgMetrics

In [19]:
# rf_model.bestModel.stages[0]

Get the feature importances of the trained model. What 5 features are the most important in the dataset?

In [20]:
# rf_model.bestModel.featureImportances

In [21]:
# predictions = rf_model.transform(training_data)
# predictions.select("Target", "prediction").show(7)

Apply model to the validation part of your set and get the accuracy score for the data. Use the MulticlassClassificationEvaluator class from the ml.evaluation module. 

In [22]:
# accuracy = evaluator.evaluate(predictions)

In [23]:
# print("Test Error = %g" % (1.0 - accuracy))
# print("Accuracy = %g" % (accuracy))

Are your results better than the results from the Logistic Regression model?

# Performing test submission

Apply the models to the test dataset.

<b>Note!</b> Dataset will be changed during the test phase. Your last cell output must be the accuracy score.

In [24]:
test_data = spark_session.read.load(
    "/data/covertype2/test.csv",
    format='com.databricks.spark.csv', 
    header=True, 
    inferSchema=True
).repartition(60)

In [25]:
predictions = rf_model.transform(test_data)
# predictions.select("Target", "prediction").show(7)

In [26]:
accuracy = evaluator.evaluate(predictions)
print (accuracy)

0.7600428494911623
